In [17]:
import json    

In [16]:
from tqdm import tqdm

In [34]:
with open('./DatabaseLuatHNvGD/kv_store_text_chunks.json', "r", encoding = "utf-8") as f:
    CHUNKS = json.load(f)


In [2]:
with open('./DatabaseLuatHNvGD/vdb_entities.json', "r", encoding = "utf-8") as f:
    ENTITIES = json.load(f)

In [19]:
with open('./DatabaseLuatHNvGD/vdb_relationships.json', "r", encoding = "utf-8") as f:
    RELATIONSHIPS = json.load(f)

In [26]:
with open('./DatabaseLuatHNvGD/vdb_chunks.json', "r", encoding = "utf-8") as f:
    VBDchunks = json.load(f)

In [22]:
with open('./data/Test /W_gold_answers/test_multi_choice.json', "r", encoding = "utf-8") as f:
    test_multi_choice = json.load(f)

In [53]:
with open('./data/Test /W_gold_answers/test_tuluan.json', "r", encoding = "utf-8") as f:
    test_tuluan = json.load(f)

In [54]:
with open('./data/Test /W_gold_answers/test_tuvan.json', "r", encoding = "utf-8") as f:
    test_tuvan = json.load(f)

In [ ]:
test_tuluan
test_tuvan

# Count chunks, tokens, words

In [35]:
len(CHUNKS)

2772

### ==> Dataset bao gồm 2772 chunks

In [36]:
type(CHUNKS)

dict

In [36]:
num_tokens = 0
num_words = 0
all_text_database = ""


for i in CHUNKS:
    #print(i)
    #print(CHUNKS[i])

    a = CHUNKS[i]
    #print(len(a))

    #print(a['tokens'])
    #print(a['content'])
    #print(a['chunk_order_index'])
    #print(a['full_doc_id'])

    num_tokens += a['tokens']
    num_words += len(a['content'].split())

    all_text_database += a['content'] + ' '


In [37]:
num_tokens

2738108

In [38]:
num_words

1949688

In [39]:
len(all_text_database.split())

1949688

In [40]:
a =all_text_database

In [41]:
n = 4
length = len(a)
part_size = length // n
parts = [a[i * part_size: (i + 1) * part_size] for i in range(n - 1)]
parts.append(a[(n - 1) * part_size:])  # Phần cuối cùng

# Lưu từng phần vào các tệp .txt
for i, part in enumerate(parts):
    file_name = f"part_{i + 1}.txt"
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(part)
    print(f"Đã lưu phần {i + 1} vào {file_name}")

print("Hoàn thành!")

Đã lưu phần 1 vào part_1.txt
Đã lưu phần 2 vào part_2.txt
Đã lưu phần 3 vào part_3.txt
Đã lưu phần 4 vào part_4.txt
Hoàn thành!


### Dataset gồm có:
- ~2,7tr tokens
- ~2tr từ

#### Important note: do quá trình xử lý data bẩn chưa hoàn thiện --> có nhiều tokens không liên quan
==> có thể tính có khoảng ~2tr tokens 

# Phân tích ENTITIES

In [3]:
len(ENTITIES)

3

In [4]:
type(ENTITIES)

dict

In [5]:
for i in ENTITIES:
    print(i)
    print(type(ENTITIES[i]))

    break

embedding_dim
<class 'int'>


In [6]:
def extract_dict_structure(d, depth=0):
    structure = {}
    indent = "  " * depth
    for key, value in d.items():
        if isinstance(value, dict):
            structure[key] = extract_dict_structure(value, depth + 1)
        elif isinstance(value, list):
            structure[key] = [f"<{type(value[0]).__name__}>" if value else "<empty_list>"]
        else:
            structure[key] = f"<{type(value).__name__}>"
    return structure

In [7]:
extract_dict_structure(ENTITIES)

{'embedding_dim': '<int>', 'data': ['<dict>'], 'matrix': '<str>'}

In [8]:
ENTITIES['embedding_dim']

1536

In [15]:
len(ENTITIES['matrix'])

117055488

In [51]:
len(ENTITIES['data'])

14289

In [18]:
ENTITIES['data'][1300]

{'__id__': 'ent-b3602c0dfcf2ce8c1105908a1a11eafa',
 'entity_name': '"TÌNH TRẠNG HÔN NHÂN"'}

### Graph có 14289 entities

# Phân tích RELATIONSHIPS

In [20]:
len(RELATIONSHIPS)

3

In [21]:
extract_dict_structure(RELATIONSHIPS)

{'embedding_dim': '<int>', 'data': ['<dict>'], 'matrix': '<str>'}

In [22]:
RELATIONSHIPS['embedding_dim']

1536

In [23]:
len(RELATIONSHIPS['data'])

15774

In [25]:
RELATIONSHIPS['data'][675]

{'__id__': 'rel-e5c012439d78f4dabc85891ccafe5b8c',
 'src_id': '"LUẬT HÔN NHÂN VÀ GIA ĐÌNH 2014"',
 'tgt_id': '"ĐIỀU 60"'}

### Graph có 15774 cạnh

### Mật độ graph = 2E/V(V-1) ~ 1,5.10^(-4) ~ 0,015%

# VBDchunks

In [27]:
len(VBDchunks)

3

In [28]:
extract_dict_structure(VBDchunks)

{'embedding_dim': '<int>', 'data': ['<dict>'], 'matrix': '<str>'}

In [29]:
VBDchunks['embedding_dim']

1536

In [30]:
len(VBDchunks['matrix'])

24444928

In [32]:
len(VBDchunks['data'])

2984

In [33]:
VBDchunks['data'][0]

{'__id__': 'chunk-da8258a848ccd42509596704aadfafe3'}

## Tóm tắt, mô tả dataset

In [ ]:
part1 = '''
Luật Hôn nhân và Gia đình năm 2014 của Việt Nam điều chỉnh các khía cạnh của hôn nhân và gia đình, bao gồm các tiêu chuẩn ứng xử và trách nhiệm của cá nhân, tổ chức, nhà nước và xã hội. Luật này dựa trên các nguyên tắc hôn nhân tự nguyện, tiến bộ, một vợ một chồng, bình đẳng vợ chồng, xây dựng gia đình hạnh phúc, bảo vệ trẻ em, người cao tuổi và người khuyết tật, và kế thừa các giá trị văn hóa truyền thống.
Kết hôn được định nghĩa là sự xác lập quan hệ vợ chồng giữa nam và nữ theo luật định. Luật quy định rõ các điều kiện kết hôn, bao gồm độ tuổi, sự tự nguyện và không thuộc các trường hợp cấm kết hôn. Những hành vi như kết hôn giả tạo, tảo hôn, cưỡng ép kết hôn, cản trở kết hôn đều bị cấm. Nhà nước không thừa nhận hôn nhân đồng giới.
Việc sống chung như vợ chồng mà không đăng ký kết hôn không làm phát sinh quyền và nghĩa vụ vợ chồng. Tuy nhiên, quyền và nghĩa vụ đối với con cái và tài sản sẽ được giải quyết theo thỏa thuận hoặc theo Bộ luật Dân sự.
Vợ chồng có quyền và nghĩa vụ bình đẳng trong gia đình. Họ có nghĩa vụ yêu thương, chung thủy, tôn trọng, quan tâm, chăm sóc và giúp đỡ lẫn nhau. Việc lựa chọn nơi cư trú, học tập, làm việc và tham gia các hoạt động xã hội do vợ chồng thỏa thuận. Luật cũng quy định về đại diện giữa vợ và chồng trong các giao dịch.
Chế độ tài sản của vợ chồng có thể theo luật định hoặc theo thỏa thuận. Luật quy định chi tiết về tài sản chung, tài sản riêng, việc chiếm hữu, sử dụng và định đoạt tài sản.
Ly hôn là việc chấm dứt quan hệ vợ chồng theo quyết định của Tòa án. Luật quy định về ly hôn thuận tình và ly hôn theo yêu cầu của một bên. Việc trông nom, chăm sóc, nuôi dưỡng và giáo dục con cái sau ly hôn được quy định chi tiết. Nguyên tắc giải quyết tài sản khi ly hôn là chia đôi tài sản chung, nhưng có tính đến các yếu tố như hoàn cảnh gia đình, công sức đóng góp và lỗi của mỗi bên.
Luật cũng quy định về quyền và nghĩa vụ giữa cha mẹ và con, giữa các thành viên khác trong gia đình, và về nghĩa vụ cấp dưỡng. Cuối cùng, luật này cũng đề cập đến việc áp dụng pháp luật đối với quan hệ hôn nhân và gia đình có yếu tố nước ngoài.
'''

In [ ]:
dataset_describe_chatgpt = '''Dataset này cung cấp thông tin chi tiết về Luật Hôn nhân và Gia đình Việt Nam năm 2014, tập trung vào các quy định pháp lý liên quan đến hôn nhân, gia đình, tài sản, quyền và nghĩa vụ của các thành viên trong gia đình. Nội dung bao gồm các nguyên tắc cơ bản như hôn nhân tự nguyện, tiến bộ, một vợ một chồng, bình đẳng giới, và bảo vệ trẻ em, người cao tuổi, người khuyết tật. Dataset giải thích các điều kiện kết hôn, những hành vi bị cấm như kết hôn giả tạo, tảo hôn, cưỡng ép kết hôn, cản trở kết hôn, đồng thời nhấn mạnh việc Nhà nước không công nhận hôn nhân đồng giới. Quy định về quan hệ vợ chồng bao gồm quyền và nghĩa vụ bình đẳng, sự thỏa thuận về nơi cư trú, học tập, làm việc, và tham gia các hoạt động xã hội. Dataset cũng làm rõ chế độ tài sản của vợ chồng theo luật định hoặc theo thỏa thuận, cách phân chia tài sản khi ly hôn, và các quyền, nghĩa vụ đối với con cái. Ngoài ra, dataset còn đề cập đến quyền và nghĩa vụ giữa cha mẹ và con, nghĩa vụ cấp dưỡng giữa các thành viên trong gia đình, và các quy định áp dụng cho quan hệ hôn nhân, gia đình có yếu tố nước ngoài. Đây là nguồn tài liệu hữu ích cho nghiên cứu pháp lý, nâng cao hiểu biết về quyền và trách nhiệm trong hôn nhân và gia đình tại Việt Nam.'''

In [ ]:
dataset_describe = '''Dataset này cung cấp thông tin toàn diện về Luật Hôn nhân và Gia đình Việt Nam, có hiệu lực từ ngày 19/06/2014, với nội dung tập trung vào các quy định pháp lý về chế độ hôn nhân và gia đình cũng như cách ứng xử giữa các thành viên. Luật nhấn mạnh các nguyên tắc như hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng, và tôn trọng sự đa dạng về dân tộc, tôn giáo, tín ngưỡng, và quốc tịch. Mục tiêu của luật là xây dựng gia đình ấm no, tiến bộ, hạnh phúc, đồng thời bảo vệ các đối tượng dễ bị tổn thương như trẻ em, người cao tuổi, và người khuyết tật, đồng thời kế thừa các giá trị văn hóa, đạo đức truyền thống tốt đẹp của dân tộc.

Dataset bao gồm các định nghĩa pháp lý quan trọng như hôn nhân, gia đình, tập quán, kết hôn, ly hôn, và các hành vi vi phạm pháp luật liên quan. Luật quy định chi tiết về trách nhiệm của Nhà nước trong việc bảo hộ hôn nhân và gia đình, nghiêm cấm các hành vi như kết hôn giả tạo, tảo hôn, cưỡng ép kết hôn, ly hôn giả tạo, bạo lực gia đình, và lợi dụng hôn nhân để trục lợi.

Các nội dung chi tiết gồm:

Điều kiện kết hôn và đăng ký kết hôn: Quy định cụ thể về độ tuổi, sự tự nguyện, và hậu quả pháp lý của việc kết hôn trái pháp luật.
Chế độ tài sản của vợ chồng: Quy định về tài sản chung, tài sản riêng, quyền và nghĩa vụ của vợ chồng trong quản lý và định đoạt tài sản.
Quyền yêu cầu ly hôn: Bao gồm hòa giải, thuận tình ly hôn, ly hôn theo yêu cầu một bên, và nguyên tắc chia tài sản khi ly hôn.
Quan hệ cha mẹ và con: Quy định về quyền, nghĩa vụ giữa cha mẹ và con, việc xác định cha mẹ con, mang thai hộ vì mục đích nhân đạo, và cấp dưỡng.
Quan hệ giữa các thành viên khác trong gia đình: Bao gồm nghĩa vụ giữa ông bà, cháu, anh chị em.
Quan hệ hôn nhân và gia đình có yếu tố nước ngoài: Hướng dẫn về việc áp dụng pháp luật, thẩm quyền giải quyết, và công nhận bản án, quyết định của nước ngoài.
Dataset được thiết kế để hỗ trợ nghiên cứu pháp lý, nâng cao nhận thức về quyền và nghĩa vụ trong hôn nhân và gia đình, và làm nguồn tham khảo hữu ích trong các hoạt động tư vấn, giáo dục pháp luật, hoặc phân tích chính sách. '''

# Tạo tập Evalution

In [27]:
PROMPT = {}

In [ ]:
PROMPT['users'] = '''Cho mô tả về dataset như sau:

## Mô Tả
Dataset này cung cấp thông tin toàn diện về Luật Hôn nhân và Gia đình Việt Nam, có hiệu lực từ ngày 19/06/2014, với nội dung tập trung vào các quy định pháp lý về chế độ hôn nhân và gia đình cũng như cách ứng xử giữa các thành viên. Luật nhấn mạnh các nguyên tắc như hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng, và tôn trọng sự đa dạng về dân tộc, tôn giáo, tín ngưỡng, và quốc tịch. Mục tiêu của luật là xây dựng gia đình ấm no, tiến bộ, hạnh phúc, đồng thời bảo vệ các đối tượng dễ bị tổn thương như trẻ em, người cao tuổi, và người khuyết tật, đồng thời kế thừa các giá trị văn hóa, đạo đức truyền thống tốt đẹp của dân tộc.

Dataset bao gồm các định nghĩa pháp lý quan trọng như hôn nhân, gia đình, tập quán, kết hôn, ly hôn, và các hành vi vi phạm pháp luật liên quan. Luật quy định chi tiết về trách nhiệm của Nhà nước trong việc bảo hộ hôn nhân và gia đình, nghiêm cấm các hành vi như kết hôn giả tạo, tảo hôn, cưỡng ép kết hôn, ly hôn giả tạo, bạo lực gia đình, và lợi dụng hôn nhân để trục lợi.

Các nội dung chi tiết gồm:

Điều kiện kết hôn và đăng ký kết hôn: Quy định cụ thể về độ tuổi, sự tự nguyện, và hậu quả pháp lý của việc kết hôn trái pháp luật.
Chế độ tài sản của vợ chồng: Quy định về tài sản chung, tài sản riêng, quyền và nghĩa vụ của vợ chồng trong quản lý và định đoạt tài sản.
Quyền yêu cầu ly hôn: Bao gồm hòa giải, thuận tình ly hôn, ly hôn theo yêu cầu một bên, và nguyên tắc chia tài sản khi ly hôn.
Quan hệ cha mẹ và con: Quy định về quyền, nghĩa vụ giữa cha mẹ và con, việc xác định cha mẹ con, mang thai hộ vì mục đích nhân đạo, và cấp dưỡng.
Quan hệ giữa các thành viên khác trong gia đình: Bao gồm nghĩa vụ giữa ông bà, cháu, anh chị em.
Quan hệ hôn nhân và gia đình có yếu tố nước ngoài: Hướng dẫn về việc áp dụng pháp luật, thẩm quyền giải quyết, và công nhận bản án, quyết định của nước ngoài.
Dataset được thiết kế để hỗ trợ nghiên cứu pháp lý, nâng cao nhận thức về quyền và nghĩa vụ trong hôn nhân và gia đình, và làm nguồn tham khảo hữu ích trong các hoạt động tư vấn, giáo dục pháp luật, hoặc phân tích chính sách. 

## Nhiệm Vụ
Hãy tạo ra 5 người dùng tiềm năng nhất (5 potential users) sẽ tham gia vào tập dữ liệu này(cố  gắng tạo ra đa dạng người dùng, từ những người bình thường đến chuyên gia). Đối với mỗi người dùng, hãy liệt kê 5 tác vụ mà họ sẽ thực hiện với tập dữ liệu này.
Hãy tạo các người dùng, tác vụ sao cho họ "hiểu biết" ở cấp độ CAO về TOÀN BỘ tập dữ liệu.

## Đầu ra
- <người dùng 1> : <mô tả người dùng>
    - <Tác vụ 1> : <Mô tả tác vụ>
    - <Tác vụ 2> : <Mô tả tác vụ>
- <người dùng 2> : <mô tả người dùng>
...

'''


### Người dùng 1: **Luật sư chuyên gia về hôn nhân và gia đình**  
- **Tác vụ 1**: Phân tích các điều khoản trong luật để tư vấn khách hàng về quyền và nghĩa vụ trong hôn nhân, ly hôn, và phân chia tài sản.  
- **Tác vụ 2**: Chuẩn bị hồ sơ pháp lý cho các vụ kiện tụng liên quan đến bạo lực gia đình hoặc tranh chấp quyền nuôi con.  
- **Tác vụ 3**: Đánh giá và xác minh tính hợp pháp của các hợp đồng hôn nhân và thỏa thuận tài sản giữa các cặp đôi.  
- **Tác vụ 4**: Giảng dạy hoặc tổ chức các buổi đào tạo cho luật sư tập sự hoặc sinh viên luật về luật hôn nhân và gia đình.  
- **Tác vụ 5**: Soạn thảo các văn bản kiến nghị sửa đổi, bổ sung luật dựa trên thực tiễn áp dụng và nhu cầu xã hội.  

---

### Người dùng 2: **Nhà nghiên cứu chính sách xã hội**  
- **Tác vụ 1**: Nghiên cứu tác động của luật đến các nhóm dễ bị tổn thương như trẻ em, phụ nữ, và người cao tuổi.  
- **Tác vụ 2**: Phân tích các quy định hiện hành để so sánh với pháp luật quốc tế và đề xuất cải cách.  
- **Tác vụ 3**: Thu thập dữ liệu từ luật để xây dựng các mô hình dự đoán xu hướng xã hội liên quan đến hôn nhân và gia đình.  
- **Tác vụ 4**: Xuất bản các bài báo học thuật về hiệu quả và hạn chế của luật hôn nhân và gia đình Việt Nam.  
- **Tác vụ 5**: Tham gia hội thảo hoặc nhóm nghiên cứu để tư vấn chính phủ trong việc hoạch định chính sách.  

---

### Người dùng 3: **Giảng viên đại học luật**  
- **Tác vụ 1**: Biên soạn giáo trình giảng dạy môn Luật Hôn nhân và Gia đình dựa trên các quy định trong dataset.  
- **Tác vụ 2**: Tổ chức các bài tập tình huống và mô phỏng các phiên tòa để sinh viên thực hành áp dụng luật.  
- **Tác vụ 3**: Hướng dẫn sinh viên thực hiện nghiên cứu chuyên sâu về các chủ đề như tài sản vợ chồng hay quyền nuôi con.  
- **Tác vụ 4**: Đánh giá hiệu quả giáo dục pháp luật thông qua các khảo sát hoặc bài kiểm tra liên quan đến luật này.  
- **Tác vụ 5**: Tổ chức hội thảo học thuật để trao đổi kiến thức với các chuyên gia pháp lý khác.  

---

### Người dùng 4: **Cán bộ công chức tại tòa án nhân dân**  
- **Tác vụ 1**: Thẩm định hồ sơ kết hôn và ly hôn, xác minh các điều kiện theo luật.  
- **Tác vụ 2**: Xét xử và ra quyết định liên quan đến phân chia tài sản, quyền nuôi con trong các vụ ly hôn.  
- **Tác vụ 3**: Tư vấn pháp lý cho các bên liên quan về các quy định của luật trong quá trình hòa giải.  
- **Tác vụ 4**: Phân tích và giải quyết các vụ án có yếu tố nước ngoài, áp dụng đúng quy định về thẩm quyền.  
- **Tác vụ 5**: Lập báo cáo thống kê về các vụ án hôn nhân và gia đình để phục vụ công tác quản lý.  

---

### Người dùng 5: **Chuyên gia tư vấn tâm lý gia đình**  
- **Tác vụ 1**: Nghiên cứu luật để hiểu rõ quyền và nghĩa vụ pháp lý của các thành viên gia đình trong các trường hợp tư vấn.  
- **Tác vụ 2**: Xây dựng các tài liệu hướng dẫn hoặc sách nhỏ về pháp luật để giáo dục các gia đình về quyền lợi của họ.  
- **Tác vụ 3**: Thực hiện các buổi tư vấn hòa giải trong các trường hợp mâu thuẫn gia đình trước khi tiến hành thủ tục pháp lý.  
- **Tác vụ 4**: Hợp tác với luật sư hoặc cơ quan pháp lý để đảm bảo lợi ích tốt nhất cho trẻ em hoặc người dễ bị tổn thương trong gia đình.  
- **Tác vụ 5**: Tư vấn cách xử lý các trường hợp ly hôn hoặc bạo lực gia đình sao cho phù hợp với quy định pháp luật và văn hóa xã hội.  

In [ ]:
PROMPT['questions'] = '''
Cho mô tả về dataset như sau:

## Mô Tả
Dataset này cung cấp thông tin toàn diện về Luật Hôn nhân và Gia đình Việt Nam, có hiệu lực từ ngày 19/06/2014, với nội dung tập trung vào các quy định pháp lý về chế độ hôn nhân và gia đình cũng như cách ứng xử giữa các thành viên. Luật nhấn mạnh các nguyên tắc như hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng, và tôn trọng sự đa dạng về dân tộc, tôn giáo, tín ngưỡng, và quốc tịch. Mục tiêu của luật là xây dựng gia đình ấm no, tiến bộ, hạnh phúc, đồng thời bảo vệ các đối tượng dễ bị tổn thương như trẻ em, người cao tuổi, và người khuyết tật, đồng thời kế thừa các giá trị văn hóa, đạo đức truyền thống tốt đẹp của dân tộc.

Dataset bao gồm các định nghĩa pháp lý quan trọng như hôn nhân, gia đình, tập quán, kết hôn, ly hôn, và các hành vi vi phạm pháp luật liên quan. Luật quy định chi tiết về trách nhiệm của Nhà nước trong việc bảo hộ hôn nhân và gia đình, nghiêm cấm các hành vi như kết hôn giả tạo, tảo hôn, cưỡng ép kết hôn, ly hôn giả tạo, bạo lực gia đình, và lợi dụng hôn nhân để trục lợi.

Các nội dung chi tiết gồm:

Điều kiện kết hôn và đăng ký kết hôn: Quy định cụ thể về độ tuổi, sự tự nguyện, và hậu quả pháp lý của việc kết hôn trái pháp luật.
Chế độ tài sản của vợ chồng: Quy định về tài sản chung, tài sản riêng, quyền và nghĩa vụ của vợ chồng trong quản lý và định đoạt tài sản.
Quyền yêu cầu ly hôn: Bao gồm hòa giải, thuận tình ly hôn, ly hôn theo yêu cầu một bên, và nguyên tắc chia tài sản khi ly hôn.
Quan hệ cha mẹ và con: Quy định về quyền, nghĩa vụ giữa cha mẹ và con, việc xác định cha mẹ con, mang thai hộ vì mục đích nhân đạo, và cấp dưỡng.
Quan hệ giữa các thành viên khác trong gia đình: Bao gồm nghĩa vụ giữa ông bà, cháu, anh chị em.
Quan hệ hôn nhân và gia đình có yếu tố nước ngoài: Hướng dẫn về việc áp dụng pháp luật, thẩm quyền giải quyết, và công nhận bản án, quyết định của nước ngoài.
Dataset được thiết kế để hỗ trợ nghiên cứu pháp lý, nâng cao nhận thức về quyền và nghĩa vụ trong hôn nhân và gia đình, và làm nguồn tham khảo hữu ích trong các hoạt động tư vấn, giáo dục pháp luật, hoặc phân tích chính sách. 

## Nhiệm Vụ
Từ 5 tác vụ được mô tả về một người dùng tiềm năng của dataset trên hãy tạo ra 5 câu hỏi cho mỗi tác vụ. Như vậy sẽ có 25 câu hỏi.
Hãy tạo các câu hỏi sao cho họ "hiểu biết" ở cấp độ CAO về TOÀN BỘ tập dữ liệu.

## Đầu ra
{
    "Tác vụ 1" : [<Câu hỏi 1>, <Câu hỏi 2>, <Câu hỏi 3>, <Câu hỏi 4>, <Câu hỏi 5>],
    "Tác vụ 2" : [<Câu hỏi 1>, <Câu hỏi 2>, <Câu hỏi 3>, <Câu hỏi 4>, <Câu hỏi 5>],
    ...
}

## Thông tin người dùng:
{{user_profile}}
'''

In [2]:
QUESTIONS = []

In [3]:
QUESTIONS

[]

In [4]:
a = {
    "Tác vụ 1": [
        "Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?",
        "Phân biệt tài sản chung và tài sản riêng của vợ chồng theo quy định pháp luật, và hậu quả pháp lý khi không phân định rõ ràng?",
        "Quyền yêu cầu ly hôn theo từng trường hợp (thuận tình, một bên yêu cầu) và nguyên tắc chia tài sản được áp dụng ra sao?",
        "Làm thế nào để tư vấn khách hàng về quyền và nghĩa vụ trong quan hệ cha mẹ và con, đặc biệt trong việc cấp dưỡng và xác định cha mẹ con?",
        "Những hành vi nào bị nghiêm cấm trong luật hôn nhân và gia đình, và cách phòng tránh các rủi ro pháp lý liên quan?"
    ],
    "Tác vụ 2": [
        "Cần lưu ý gì khi thu thập và chuẩn bị chứng cứ cho một vụ kiện liên quan đến bạo lực gia đình?",
        "Làm thế nào để xây dựng lập luận pháp lý bảo vệ quyền lợi của khách hàng trong tranh chấp quyền nuôi con?",
        "Quy trình hòa giải trong các vụ kiện hôn nhân và gia đình được quy định như thế nào và vai trò của luật sư trong quá trình này?",
        "Các quy định pháp luật liên quan đến bảo vệ quyền lợi của trẻ em trong các vụ ly hôn và bạo lực gia đình là gì?",
        "Làm thế nào để xác định trách nhiệm của các bên trong vụ kiện tranh chấp liên quan đến cấp dưỡng hoặc mang thai hộ vì mục đích nhân đạo?"
    ],
    "Tác vụ 3": [
        "Những điều khoản nào cần có trong hợp đồng hôn nhân để đảm bảo tính hợp pháp và quyền lợi của các bên?",
        "Làm thế nào để xác minh thỏa thuận tài sản của vợ chồng có tuân thủ đúng quy định pháp luật hay không?",
        "Các yếu tố nào có thể khiến một hợp đồng hôn nhân bị vô hiệu và cách phòng tránh?",
        "Phân tích quy định pháp luật về việc quản lý và định đoạt tài sản chung, tài sản riêng trong hôn nhân?",
        "Hậu quả pháp lý của việc không đăng ký kết hôn khi thỏa thuận tài sản giữa các bên phát sinh tranh chấp là gì?"
    ],
    "Tác vụ 4": [
        "Làm thế nào để trình bày một cách toàn diện về nguyên tắc hôn nhân tự nguyện, tiến bộ, một vợ một chồng, và bình đẳng?",
        "Những phương pháp giảng dạy hiệu quả để sinh viên luật hiểu rõ các hành vi bị nghiêm cấm trong luật hôn nhân và gia đình?",
        "Các điểm nổi bật trong quy định pháp luật về quyền và nghĩa vụ giữa cha mẹ và con cần được nhấn mạnh trong bài giảng?",
        "Cách tiếp cận hiệu quả khi hướng dẫn luật sư tập sự về các thủ tục pháp lý liên quan đến đăng ký kết hôn hoặc ly hôn?",
        "Làm thế nào để tổ chức một buổi thực hành phân tích các tình huống pháp lý phức tạp về chế độ tài sản của vợ chồng?"
    ],
    "Tác vụ 5": [
        "Những hạn chế nào trong luật hôn nhân và gia đình hiện hành cần được sửa đổi để phù hợp hơn với thực tiễn xã hội?",
        "Cách thu thập dữ liệu thực tiễn và phản hồi từ cộng đồng để xây dựng các kiến nghị sửa đổi luật?",
        "Làm thế nào để đảm bảo các kiến nghị sửa đổi luật vừa bảo vệ quyền lợi của cá nhân vừa duy trì các giá trị truyền thống tốt đẹp?",
        "Các thách thức pháp lý nào phát sinh từ quan hệ hôn nhân và gia đình có yếu tố nước ngoài cần được giải quyết trong luật?",
        "Những nguyên tắc pháp lý nào cần được bổ sung để tăng cường bảo vệ các đối tượng dễ bị tổn thương như trẻ em và người cao tuổi?"
    ]
}


for i in a:
    l = a[i]

    for q in l:
        QUESTIONS.append(q)

In [5]:
len(QUESTIONS)

25

In [6]:
a = {
    "Tác vụ 1": [
        "Luật Hôn nhân và Gia đình quy định cụ thể thế nào về việc bảo vệ trẻ em trong gia đình có yếu tố bạo lực hoặc ly hôn?",
        "Các điều kiện kết hôn và hạn chế hôn nhân trong luật có ảnh hưởng gì đến việc giảm thiểu tảo hôn và cưỡng ép hôn nhân?",
        "Luật đã đề cập như thế nào đến quyền lợi và trách nhiệm của người cao tuổi trong mối quan hệ gia đình, và điều này có tác động ra sao đến chất lượng cuộc sống của họ?",
        "Chế độ tài sản chung của vợ chồng được quy định trong luật có đảm bảo quyền lợi bình đẳng của phụ nữ không? Nếu có, hãy phân tích cơ chế bảo vệ này.",
        "Làm thế nào luật này hỗ trợ mang thai hộ vì mục đích nhân đạo mà vẫn bảo vệ quyền lợi của trẻ em và người mang thai hộ?"
    ],
    "Tác vụ 2": [
        "So sánh quy định về điều kiện kết hôn trong Luật Hôn nhân và Gia đình Việt Nam với các tiêu chuẩn quốc tế, ví dụ như Công ước Liên Hợp Quốc về Quyền Trẻ em.",
        "Phân tích cách tiếp cận của luật Việt Nam đối với quyền ly hôn theo yêu cầu một bên so với các quốc gia có hệ thống pháp luật tương tự.",
        "Những điểm khác biệt lớn nhất giữa quy định về bạo lực gia đình trong luật Việt Nam và pháp luật quốc tế là gì?",
        "Các quy định về hôn nhân có yếu tố nước ngoài trong luật Việt Nam có đảm bảo quyền lợi của công dân Việt Nam so với các quốc gia khác không?",
        "Hệ thống pháp lý của Việt Nam về chế độ tài sản vợ chồng có tương đồng hoặc khác biệt gì so với luật pháp ở các nước phát triển?"
    ],
    "Tác vụ 3": [
        "Dựa trên các quy định về quyền yêu cầu ly hôn, làm thế nào để xây dựng mô hình dự đoán tỷ lệ ly hôn ở các nhóm dân số khác nhau?",
        "Luật quy định gì về quan hệ giữa các thành viên trong gia đình, và dữ liệu này có thể sử dụng để phân tích xu hướng phân bổ trách nhiệm trong gia đình không?",
        "Các quy định về mang thai hộ có thể được tích hợp như thế nào vào mô hình dự đoán xu hướng xã hội liên quan đến sự chấp nhận của cộng đồng?",
        "Dữ liệu về chế độ tài sản vợ chồng trong luật có thể hỗ trợ phân tích kinh tế gia đình ở mức độ nào?",
        "Các yếu tố pháp lý được xác định trong luật có thể dự đoán khả năng xuất hiện các tranh chấp gia đình trong tương lai không? Nếu có, làm thế nào?"
    ],
    "Tác vụ 4": [
        "Hiệu quả của quy định về hòa giải trong ly hôn đã được đánh giá như thế nào, và đâu là hạn chế của cơ chế này?",
        "Hành vi bạo lực gia đình bị nghiêm cấm trong luật, nhưng việc thực thi thực tế có đạt hiệu quả không? Phân tích nguyên nhân nếu có sự khác biệt.",
        "Các quy định về tài sản riêng và tài sản chung của vợ chồng trong luật có ảnh hưởng thế nào đến việc thúc đẩy bình đẳng giới?",
        "Hãy đánh giá cách thức luật Việt Nam điều chỉnh các quan hệ hôn nhân có yếu tố nước ngoài và hiệu quả trong việc bảo vệ quyền lợi của người Việt Nam.",
        "Các nguyên tắc của Luật Hôn nhân và Gia đình Việt Nam có phù hợp với các giá trị đạo đức truyền thống và yêu cầu hiện đại không? Hãy phân tích chi tiết."
    ],
    "Tác vụ 5": [
        "Những điểm nào trong Luật Hôn nhân và Gia đình cần sửa đổi để phù hợp với sự phát triển của xã hội Việt Nam hiện đại?",
        "Làm thế nào để tư vấn chính phủ cải thiện các quy định về bảo vệ trẻ em trong các gia đình có nguy cơ ly hôn hoặc bạo lực?",
        "Các nguyên tắc cơ bản của luật có thể áp dụng vào việc xây dựng chính sách hỗ trợ hôn nhân đa văn hóa như thế nào?",
        "Trong các hội thảo về cải cách luật pháp, làm thế nào để lập luận thuyết phục rằng quy định về tảo hôn cần được giám sát và thực thi mạnh mẽ hơn?",
        "Việc thực hiện quy định về chế độ tài sản chung của vợ chồng cần điều chỉnh ra sao để đảm bảo công bằng xã hội và tăng cường sự đồng thuận từ cộng đồng?"
    ]
}

for i in a:
    l = a[i]

    for q in l:
        QUESTIONS.append(q)

In [7]:
a = {
    "Tác vụ 1": [
        "Các nguyên tắc cơ bản nào của Luật Hôn nhân và Gia đình cần được nhấn mạnh khi biên soạn giáo trình giảng dạy?",
        "Làm thế nào để tổ chức nội dung về quyền và nghĩa vụ của cha mẹ và con một cách logic và dễ hiểu trong giáo trình?",
        "Quy định về chế độ tài sản vợ chồng có thể được minh họa qua các ví dụ thực tế như thế nào để giúp sinh viên dễ nắm bắt?",
        "Nên sắp xếp nội dung về quan hệ hôn nhân có yếu tố nước ngoài như thế nào trong giáo trình để tạo sự liên kết với các quy định trong nước?",
        "Có những phương pháp nào để biên soạn các bài tập cuối chương dựa trên các hành vi bị nghiêm cấm trong luật?"
    ],
    "Tác vụ 2": [
        "Làm thế nào để thiết kế tình huống thực tế mô phỏng việc giải quyết ly hôn theo yêu cầu của một bên?",
        "Các bước cần thiết để tổ chức phiên tòa mô phỏng liên quan đến việc phân chia tài sản chung sau ly hôn là gì?",
        "Những yếu tố pháp lý nào cần xem xét khi xây dựng tình huống mô phỏng về quyền nuôi con sau ly hôn?",
        "Quy trình mô phỏng xử lý hành vi bạo lực gia đình và vai trò của luật sư trong việc bảo vệ quyền lợi của các bên là gì?",
        "Làm thế nào để đánh giá năng lực sinh viên qua các tình huống liên quan đến quan hệ hôn nhân có yếu tố nước ngoài?"
    ],
    "Tác vụ 3": [
        "Các yếu tố nào trong chế độ tài sản của vợ chồng là chủ đề nghiên cứu tiềm năng cho sinh viên?",
        "Quy định pháp luật nào hỗ trợ nghiên cứu về quyền và nghĩa vụ giữa cha mẹ và con trong các tình huống đặc biệt?",
        "Những khía cạnh nào cần phân tích khi nghiên cứu về việc mang thai hộ vì mục đích nhân đạo?",
        "Các yếu tố văn hóa và pháp lý nào ảnh hưởng đến nghiên cứu về quan hệ hôn nhân và gia đình có yếu tố nước ngoài?",
        "Làm thế nào để hướng dẫn sinh viên phân tích tác động của các hành vi vi phạm pháp luật đến quyền lợi của các bên liên quan?"
    ],
    "Tác vụ 4": [
        "Những tiêu chí nào cần thiết để đánh giá hiệu quả của giáo dục pháp luật về các điều kiện kết hôn và đăng ký kết hôn?",
        "Làm thế nào để xây dựng khảo sát đo lường nhận thức của sinh viên về các quy định pháp luật liên quan đến ly hôn?",
        "Phương pháp nào hữu hiệu nhất để kiểm tra khả năng hiểu và áp dụng luật trong các trường hợp về chế độ tài sản vợ chồng?",
        "Các bài kiểm tra nên tập trung vào những khía cạnh nào để đánh giá kỹ năng phân tích pháp lý về quan hệ cha mẹ và con?",
        "Làm thế nào để xác định mức độ hiểu biết của sinh viên về các quy định pháp luật đối với hôn nhân có yếu tố nước ngoài?"
    ],
    "Tác vụ 5": [
        "Những chủ đề nào trong luật này có thể trở thành điểm nhấn trong hội thảo học thuật với các chuyên gia pháp lý?",
        "Làm thế nào để thiết kế các buổi thảo luận chuyên sâu về các hành vi bị nghiêm cấm trong hôn nhân và gia đình?",
        "Các phương pháp nào hữu hiệu để trao đổi kiến thức về chế độ tài sản vợ chồng giữa các chuyên gia trong hội thảo?",
        "Hội thảo nên tập trung vào các khía cạnh nào khi trao đổi về quyền và nghĩa vụ giữa các thành viên trong gia đình?",
        "Làm thế nào để tích hợp quan điểm pháp lý quốc tế vào việc thảo luận về quan hệ hôn nhân và gia đình có yếu tố nước ngoài?"
    ]
}

for i in a:
    l = a[i]

    for q in l:
        QUESTIONS.append(q)


In [8]:
a= {
    "Tác vụ 1": [
        "Những điều kiện cần thiết để một cặp đôi được đăng ký kết hôn theo luật hôn nhân hiện hành là gì? Trường hợp ngoại lệ nào có thể áp dụng?",
        "Làm thế nào để xác minh rằng việc kết hôn là tự nguyện và không bị ép buộc hoặc lừa dối?",
        "Trong trường hợp kết hôn trái pháp luật, quy trình xử lý và hậu quả pháp lý sẽ được thực hiện như thế nào?",
        "Các tiêu chí cụ thể để xác định một hồ sơ ly hôn hợp lệ và đủ điều kiện để tiếp nhận là gì?",
        "Các biện pháp bảo vệ nào được áp dụng để xử lý trường hợp bạo lực gia đình hoặc lạm dụng trong quá trình thẩm định hồ sơ ly hôn?"
    ],
    "Tác vụ 2": [
        "Theo luật, nguyên tắc nào được áp dụng trong việc phân chia tài sản chung và tài sản riêng của vợ chồng khi ly hôn?",
        "Cơ sở pháp lý nào để quyết định quyền nuôi con trong trường hợp cả hai bên đều muốn giành quyền nuôi con?",
        "Các biện pháp bảo vệ quyền lợi cho trẻ em khi cha mẹ ly hôn được quy định cụ thể như thế nào trong luật?",
        "Khi xảy ra tranh chấp về tài sản, làm thế nào để xác minh và định giá tài sản chung và riêng của vợ chồng?",
        "Quy định nào áp dụng trong trường hợp một bên không đồng ý với quyết định phân chia tài sản hoặc quyền nuôi con của tòa án?"
    ],
    "Tác vụ 3": [
        "Những điểm nào cần nhấn mạnh khi tư vấn về quyền và nghĩa vụ của vợ chồng theo luật hôn nhân hiện hành?",
        "Trong quá trình hòa giải, các nguyên tắc nào cần tuân thủ để đảm bảo quyền lợi hợp pháp của cả hai bên?",
        "Cách tiếp cận hợp lý để giải quyết các tranh chấp về cấp dưỡng giữa cha mẹ và con cái theo luật là gì?",
        "Khi tư vấn về ly hôn thuận tình, những nội dung nào cần được làm rõ để đảm bảo không xảy ra tranh chấp sau này?",
        "Quy trình tư vấn về việc mang thai hộ vì mục đích nhân đạo theo quy định hiện hành như thế nào?"
    ],
    "Tác vụ 4": [
        "Những quy định nào áp dụng trong việc giải quyết các vụ án hôn nhân có yếu tố nước ngoài?",
        "Làm thế nào để xác định thẩm quyền xét xử khi một trong hai bên là người nước ngoài?",
        "Cách thức công nhận và thi hành bản án ly hôn do tòa án nước ngoài ban hành theo luật Việt Nam là gì?",
        "Quy định pháp lý nào được áp dụng để giải quyết tranh chấp tài sản trong các vụ án hôn nhân có yếu tố quốc tế?",
        "Các lưu ý đặc biệt khi giải quyết vấn đề quyền nuôi con trong các trường hợp liên quan đến yếu tố nước ngoài là gì?"
    ],
    "Tác vụ 5": [
        "Những thông tin nào cần thu thập để lập báo cáo thống kê về các vụ án hôn nhân và gia đình?",
        "Cách phân loại và trình bày số liệu về các vụ ly hôn, kết hôn trái pháp luật, và bạo lực gia đình trong báo cáo là gì?",
        "Các tiêu chí đánh giá hiệu quả của tòa án trong việc xử lý các vụ án hôn nhân và gia đình là gì?",
        "Làm thế nào để phân tích xu hướng các vụ án hôn nhân và gia đình dựa trên các yếu tố xã hội, kinh tế, và địa lý?",
        "Quy trình chuẩn bị báo cáo để đề xuất chính sách cải thiện công tác quản lý hôn nhân và gia đình là gì?"
    ]
}

for i in a:
    l = a[i]

    for q in l:
        QUESTIONS.append(q)


In [9]:
a= {
    "Tác vụ 1": [
        "Theo Luật Hôn nhân và Gia đình Việt Nam, quyền và nghĩa vụ của cha mẹ đối với con cái được quy định như thế nào trong các trường hợp đặc biệt như ly hôn hoặc tranh chấp quyền nuôi con?",
        "Những nguyên tắc pháp lý nào cần được tuân thủ trong việc xác định cha mẹ con và trách nhiệm cấp dưỡng trong gia đình?",
        "Các quyền và nghĩa vụ pháp lý giữa ông bà và cháu được quy định ra sao trong Luật Hôn nhân và Gia đình?",
        "Làm thế nào để phân biệt tài sản chung và tài sản riêng của vợ chồng theo luật, và các yếu tố này ảnh hưởng gì đến trách nhiệm quản lý tài sản trong gia đình?",
        "Luật nghiêm cấm những hành vi nào liên quan đến hôn nhân và gia đình, và các biện pháp pháp lý được áp dụng để xử lý vi phạm?"
    ],
    "Tác vụ 2": [
        "Làm thế nào để trình bày một cách hiệu quả về các điều kiện kết hôn và đăng ký kết hôn nhằm nâng cao nhận thức của các cặp đôi sắp cưới?",
        "Những nội dung chính nào cần được đưa vào tài liệu giáo dục về quyền và nghĩa vụ của vợ chồng đối với tài sản chung và riêng?",
        "Các ví dụ thực tế nào có thể sử dụng để minh họa trách nhiệm pháp lý giữa cha mẹ và con, nhằm giúp gia đình hiểu rõ hơn về quyền lợi của trẻ em?",
        "Tài liệu hướng dẫn pháp luật cần bao gồm những điểm quan trọng gì để giúp các gia đình giải quyết mâu thuẫn mà không cần đến sự can thiệp của pháp luật?",
        "Làm thế nào để thiết kế tài liệu giáo dục pháp luật phù hợp với các gia đình có yếu tố nước ngoài, đặc biệt về áp dụng pháp luật và công nhận quyết định từ nước ngoài?"
    ],
    "Tác vụ 3": [
        "Quy trình hòa giải trong mâu thuẫn gia đình theo quy định pháp luật bao gồm những bước cụ thể nào, và vai trò của chuyên gia tư vấn là gì?",
        "Các yếu tố pháp lý nào cần được cân nhắc khi hòa giải các mâu thuẫn về phân chia tài sản chung của vợ chồng?",
        "Làm thế nào để áp dụng Luật Hôn nhân và Gia đình trong việc giải quyết các tranh chấp liên quan đến quyền nuôi con sau ly hôn?",
        "Những nguyên tắc pháp luật nào cần được nhấn mạnh trong quá trình hòa giải các trường hợp bạo lực gia đình?",
        "Vai trò của tập quán và văn hóa truyền thống trong hòa giải gia đình được pháp luật quy định ra sao?"
    ],
    "Tác vụ 4": [
        "Những quy định pháp luật nào đảm bảo quyền lợi của trẻ em trong các trường hợp ly hôn hoặc bạo lực gia đình?",
        "Vai trò của chuyên gia tư vấn trong việc phối hợp với luật sư để bảo vệ các đối tượng dễ bị tổn thương trong gia đình là gì?",
        "Làm thế nào để áp dụng Luật Hôn nhân và Gia đình vào việc hỗ trợ người cao tuổi trong các tranh chấp gia đình liên quan đến quyền thừa kế?",
        "Quy định pháp luật nào liên quan đến mang thai hộ vì mục đích nhân đạo, và làm thế nào để đảm bảo quyền lợi của các bên liên quan?",
        "Những điểm pháp lý nào cần được xem xét khi hỗ trợ các gia đình có yếu tố nước ngoài bảo vệ quyền lợi của các thành viên?"
    ],
    "Tác vụ 5": [
        "Các bước cần thực hiện để tư vấn cho các cặp đôi về thủ tục ly hôn thuận tình theo quy định pháp luật?",
        "Làm thế nào để hỗ trợ các nạn nhân bạo lực gia đình tuân thủ pháp luật và đảm bảo an toàn trong quá trình tố cáo?",
        "Những nguyên tắc pháp lý nào cần được chú trọng khi tư vấn phân chia tài sản trong quá trình ly hôn?",
        "Luật quy định ra sao về việc bảo vệ quyền lợi của trẻ em trong các trường hợp ly hôn, và làm thế nào để tư vấn phụ huynh đảm bảo quyền này?",
        "Các phương án tư vấn nào phù hợp với văn hóa xã hội và pháp luật để giúp các gia đình xử lý các mâu thuẫn gia đình mà không làm tổn thương các thành viên?"
    ]
}

for i in a:
    l = a[i]

    for q in l:
        QUESTIONS.append(q)


In [10]:
len(QUESTIONS)

125

# Câu trả lời từ LightRAG, mặc định sẽ để chế độ HYBRID

In [11]:
import os
from lightrag import LightRAG, QueryParam
from lightrag.llm import gpt_4o_mini_complete

import nest_asyncio
nest_asyncio.apply()

WORKING_DIR = "./DatabaseLuatHNvGD"

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

In [12]:
from dotenv import load_dotenv
import os

# Tải các biến từ tệp .env
load_dotenv()

True

In [13]:
rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=gpt_4o_mini_complete,  # Use gpt_4o_mini_complete LLM model
    # llm_model_func=gpt_4o_complete  # Optionally, use a stronger model
)

INFO:lightrag:Logger initialized for working directory: ./DatabaseLuatHNvGD
INFO:lightrag:Load KV llm_response_cache with 4 data
INFO:lightrag:Load KV full_docs with 751 data
INFO:lightrag:Load KV text_chunks with 2772 data
INFO:lightrag:Loaded graph from ./DatabaseLuatHNvGD/graph_chunk_entity_relation.graphml with 14447 nodes, 16196 edges
INFO:nano-vectordb:Load (14289, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './DatabaseLuatHNvGD/vdb_entities.json'} 14289 data
INFO:nano-vectordb:Load (7, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './DatabaseLuatHNvGD/vdb_relationships.json'} 15774 data
INFO:nano-vectordb:Load (2984, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './DatabaseLuatHNvGD/vdb_chunks.json'} 2984 data


In [55]:
q_test = "Các phương án tư vấn nào phù hợp với văn hóa xã hội và pháp luật để giúp các gia đình xử lý các mâu thuẫn gia đình mà không làm tổn thương các thành viên?"

In [58]:
res_test = rag.query(q_test, param=QueryParam(mode="hybrid"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:kw_prompt result:


{
  "high_level_keywords": ["Tư vấn", "Văn hóa xã hội", "Pháp luật", "Mâu thuẫn gia đình", "Bảo vệ thành viên"],
  "low_level_keywords": ["Phương án tư vấn", "Gia đình", "Xử lý mâu thuẫn", "Tổn thương", "Thành viên"]
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 60 entites, 77 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Global query uses 12 entites, 7 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [60]:
print(res_test)

### Giới thiệu về mâu thuẫn gia đình

Mâu thuẫn gia đình là một phần của cuộc sống thường nhật trong bất kỳ gia đình nào, và việc xử lý những mâu thuẫn này một cách nhạy cảm và hiệu quả là rất quan trọng. Điều này không chỉ giúp duy trì hòa khí trong gia đình mà còn bảo vệ tinh thần và sức khỏe của các thành viên trong gia đình. Các phương án tư vấn phù hợp với văn hóa xã hội và pháp luật hiện nay bao gồm các biện pháp hòa giải, tư vấn tâm lý và luật sư.

### 1. Hòa giải

Hòa giải là một phương thức hữu hiệu để giải quyết mâu thuẫn, giúp các bên tìm ra tiếng nói chung mà không cần phải thông qua thủ tục pháp lý nặng nề. Dưới đây là một số hình thức hòa giải:

- **Hòa giải cấp cơ sở**: Là hoạt động do các tổ chức xã hội thực hiện, tại đó, các thành viên gia đình có thể trao đổi và tìm ra giải pháp phù hợp với tất cả mọi người. Hội Liên hiệp Phụ nữ Việt Nam hay các tổ chức xã hội khác có vai trò hỗ trợ trong việc này.

- **Hòa giải qua các trung tâm tư vấn**: Nhiều trung tâm tư vấn chuyê

In [61]:
res_test = rag.query(q_test, param=QueryParam(mode="naive"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 3 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [63]:
print(res_test)

## Các Phương Án Tư Vấn Phù Hợp Với Văn Hóa Xã Hội và Pháp Luật Để Giúp Xử Lý Mâu Thuẫn Gia Đình

Trong bối cảnh xã hội hiện đại, việc xử lý mâu thuẫn gia đình một cách hiệu quả và tôn trọng văn hóa là rất quan trọng. Dưới đây là một số phương án tư vấn hợp lý, phù hợp với pháp luật và đặc thù văn hóa xã hội Việt Nam.

### 1. Hòa Giải Cấp Cơ Sở

Hòa giải là một phương thức truyền thống trong văn hóa Việt Nam, thường được áp dụng để giải quyết mâu thuẫn mà không gây tổn thương cho các thành viên. Theo pháp luật, quy trình hòa giải có thể được thực hiện tại cấp cơ sở, khuyến khích sự tham gia của những người có uy tín trong cộng đồng để tạo nên một không gian an toàn và thấu hiểu.

- **Lợi ích**: 
  - Bảo tồn bản sắc văn hóa và tập quán của cộng đồng.
  - Tăng cường sự đồng thuận và tin tưởng giữa các bên liên quan.
  - Tạo điều kiện cho mọi người tham gia một cách tích cực.

### 2. Sự Tham Gia của Người Có Uy Tín và Chức Sắc Tôn Giáo

Sự tham gia của người có uy tín trong cộng đồng hay 

In [66]:
LIGHTRAG_HYBRID_QA = []

In [68]:
for q in tqdm(QUESTIONS):
    a = {}
    a['q'] = q

    res = rag.query(q, param=QueryParam(mode="hybrid"))

    a['a'] = res

    LIGHTRAG_HYBRID_QA.append(a)

    print(a)
    print()


  0%|          | 0/125 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:kw_prompt result:


{
  "high_level_keywords": ["Điều kiện kết hôn", "Luật hôn nhân", "Tư vấn khách hàng", "Vi phạm pháp luật"],
  "low_level_keywords": ["Đối tượng kết hôn", "Giấy tờ cần thiết", "Tuổi tối thiểu", "Quyền lợi và nghĩa vụ", "Điều kiện pháp lý"]
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 60 entites, 104 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Global query uses 12 entites, 7 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  1%|          | 1/125 [00:13<28:46, 13.92s/it]

{'q': 'Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?', 'a': 'Theo quy định trong Luật Hôn nhân và Gia đình 2014, có một số điều kiện cần thiết để thực hiện việc kết hôn mà các bên cần lưu ý nhằm tránh vi phạm pháp luật. Dưới đây là những điểm chính:\n\n### 1. Đối tượng kết hôn\nNguyên tắc đầu tiên là cả hai bên phải là cá nhân đủ tuổi kết hôn, theo đó, nam phải từ đủ 20 tuổi và nữ phải từ đủ 18 tuổi. Đây là điều kiện cơ bản và quan trọng nhất, nếu không đáp ứng yêu cầu về độ tuổi, việc kết hôn sẽ không hợp pháp.\n\n### 2. Tự nguyện\nKết hôn phải được thực hiện trên cơ sở sự tự nguyện của cả hai bên. Bất kỳ hình thức ép buộc nào sẽ dẫn đến việc doanh từ mọi hiệu lực pháp lý. Các bên cần đảm bảo rằng không có sự uy hiếp hoặc ép buộc từ phía gia đình hoặc xã hội trong quyết định kết hôn.\n\n### 3. Tình trạng hôn nhân\nMột bên không được phép kết hôn trong khi vẫn đang có vợ hoặc chồng. Điều này được quy định tại Đi

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:kw_prompt result:


...

{'q': 'Những nguyên tắc pháp lý nào cần được chú trọng khi tư vấn phân chia tài sản trong quá trình ly hôn?', 'a': '## Nguyên tắc pháp lý trong phân chia tài sản khi ly hôn\n\n### 1. **Luật Hôn Nhân và Gia Đình**\nLuật Hôn nhân và Gia đình 2014 của Việt Nam là cơ sở pháp lý chính quy định về quyền và nghĩa vụ của vợ chồng liên quan đến tài sản trong trường hợp ly hôn. Luật này xác định rõ dòng chảy của tài sản chung và tài sản riêng, giúp phân định chính xác quyền lợi của mỗi bên.\n\n### 2. **Thỏa thuận về tài sản**\nMột trong những nguyên tắc quan trọng là các bên có quyền thỏa thuận về việc phân chia tài sản chung. Điều 48 của Luật Hôn nhân và Gia đình 2014 quy định rõ nội dung của thỏa thuận chế độ tài sản giữa vợ chồng, thể hiện quyền tự do lập hôn ước và tính tự nguyện trong việc phân chia tài sản.\n\n### 3. **Phân chia tài sản chung và tài sản riêng**\nLuật quy định tài sản được hình thành trong hôn nhân là tài sản chung, trong khi tài sản riêng là những tài sản mà mỗi bên sở hữu

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:kw_prompt result:


{
  "high_level_keywords": ["Bảo vệ quyền lợi trẻ em", "Ly hôn", "Tư vấn phụ huynh"],
  "low_level_keywords": ["Luật", "Quyền lợi", "Trường hợp ly hôn", "Đảm bảo quyền", "Hỗ trợ gia đình"]
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 60 entites, 108 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Global query uses 12 entites, 7 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|██████████| 125/125 [34:10<00:00, 16.40s/it]

{'q': 'Luật quy định ra sao về việc bảo vệ quyền lợi của trẻ em trong các trường hợp ly hôn, và làm thế nào để tư vấn phụ huynh đảm bảo quyền này?', 'a': '### Quy định về bảo vệ quyền lợi của trẻ em trong ly hôn\n\nLuật Hôn nhân và Gia đình năm 2014 của Việt Nam đưa ra những quy định rõ ràng nhằm bảo vệ quyền lợi của trẻ em trong các trường hợp ly hôn. Theo Điều 59 của luật này, khi phân chia tài sản chung giữa vợ và chồng, Tòa án sẽ xem xét và bảo vệ quyền lợi hợp pháp của trẻ em, đặc biệt là trẻ chưa thành niên hoặc những người đã thành niên nhưng mất năng lực hành vi dân sự. Quy định này nhấn mạnh rằng:\n\n1. **Bảo vệ lợi ích chính đáng**: Tòa án phải đảm bảo rằng quyền lợi của trẻ em sẽ được xem xét và bảo vệ, đặc biệt trong quan hệ tài sản giữa vợ chồng cũng như quyền nuôi dưỡng và chăm sóc trẻ.\n   \n2. **Nghĩa vụ cấp dưỡng**: Luật cũng quy định nghĩa vụ của cha mẹ trong việc cấp dưỡng cho con cái sau ly hôn, đảm bảo rằng dù cha mẹ không còn chung sống, trẻ vẫn được hỗ trợ về tài

In [69]:
len(LIGHTRAG_HYBRID_QA)

125

In [70]:
LIGHTRAG_HYBRID_QA[124]

{'q': 'Các phương án tư vấn nào phù hợp với văn hóa xã hội và pháp luật để giúp các gia đình xử lý các mâu thuẫn gia đình mà không làm tổn thương các thành viên?',
 'a': '### Giới thiệu về mâu thuẫn gia đình\n\nMâu thuẫn gia đình là một phần của cuộc sống thường nhật trong bất kỳ gia đình nào, và việc xử lý những mâu thuẫn này một cách nhạy cảm và hiệu quả là rất quan trọng. Điều này không chỉ giúp duy trì hòa khí trong gia đình mà còn bảo vệ tinh thần và sức khỏe của các thành viên trong gia đình. Các phương án tư vấn phù hợp với văn hóa xã hội và pháp luật hiện nay bao gồm các biện pháp hòa giải, tư vấn tâm lý và luật sư.\n\n### 1. Hòa giải\n\nHòa giải là một phương thức hữu hiệu để giải quyết mâu thuẫn, giúp các bên tìm ra tiếng nói chung mà không cần phải thông qua thủ tục pháp lý nặng nề. Dưới đây là một số hình thức hòa giải:\n\n- **Hòa giải cấp cơ sở**: Là hoạt động do các tổ chức xã hội thực hiện, tại đó, các thành viên gia đình có thể trao đổi và tìm ra giải pháp phù hợp với t

In [72]:
with open("LIGHTRAG_HYBRID_QA.json", "w", encoding="utf-8") as json_file:
    json.dump(LIGHTRAG_HYBRID_QA, json_file, ensure_ascii=False, indent=4)

# Câu trả lời Naive RAG

In [18]:
LIGHTRAG_NAIVE_QA = []

In [19]:
for q in tqdm(QUESTIONS):
    a = {}
    a['q'] = q

    res = rag.query(q, param=QueryParam(mode="naive"))

    a['a'] = res

    LIGHTRAG_NAIVE_QA.append(a)

    print(a)
    print()

  0%|          | 0/125 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 3 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  1%|          | 1/125 [00:59<2:02:32, 59.29s/it]

{'q': 'Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?', 'a': '### Các Điều Kiện Kết Hôn Theo Quy Định Của Pháp Luật Việt Nam\n\nKhi tư vấn về các điều kiện kết hôn, rất quan trọng để khách hàng hiểu rõ các quy định của Luật Hôn nhân và Gia đình, nhằm tránh các vấn đề pháp lý liên quan đến việc kết hôn trái pháp luật. Dưới đây là một số điểm cần lưu ý:\n\n#### 1. Điều Kiện Độ Tuổi\n\nLuật quy định rõ ràng về độ tuổi kết hôn. Cụ thể:\n\n- **Nam**: Từ đủ **20 tuổi** trở lên.\n- **Nữ**: Từ đủ **18 tuổi** trở lên.\n\nKhách hàng cần lưu ý rằng việc kết hôn trước độ tuổi quy định là vi phạm pháp luật và có thể bị xử lý.\n\n#### 2. Tự Nguyện Trong Quyết Định Kết Hôn\n\nMột trong những nguyên tắc cơ bản là:\n\n- **Sự tự nguyện**: Việc kết hôn phải xuất phát từ quyết định tự nguyện của cả hai bên, không bị ép buộc, cưỡng chế hay lừa dối.\n\nKhách hàng cần đảm bảo rằng cả hai bên đều đồng ý và không chịu bất kỳ áp lực nào.\

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 6 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  2%|▏         | 2/125 [01:09<1:02:01, 30.26s/it]

{'q': 'Phân biệt tài sản chung và tài sản riêng của vợ chồng theo quy định pháp luật, và hậu quả pháp lý khi không phân định rõ ràng?', 'a': '### Phân Biệt Tài Sản Chung và Tài Sản Riêng của Vợ Chồng\n\nTheo quy định của pháp luật Việt Nam, tài sản chung và tài sản riêng của vợ chồng được phân định rõ ràng trong Luật Hôn nhân và Gia đình 2014. Tài sản chung của vợ chồng là tài sản được hình thành trong thời kỳ hôn nhân, bao gồm tài sản mà hai bên cùng nhau tạo ra từ lao động, kinh doanh, hoặc tài sản mà một bên nhận được qua thừa kế hoặc quà tặng nhưng được sử dụng cho mục đích chung của gia đình.\n\nNgược lại, tài sản riêng của vợ chồng bao gồm tài sản mà mỗi bên sở hữu trước khi kết hôn, tài sản được thừa kế hoặc tặng cho cá nhân trong thời kỳ hôn nhân (nếu có sự thỏa thuận về việc này), và những tài sản mà một bên sở hữu theo quy định của pháp luật khác, cũng như lợi tức từ những tài sản riêng đó.\n\n### Hậu Quả Pháp Lý khi Không Phân Định Rõ Ràng\n\nKhi không phân định rõ ràng giữa

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 4 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  2%|▏         | 3/125 [01:18<42:15, 20.79s/it]  

...

{'q': 'Làm thế nào để hỗ trợ các nạn nhân bạo lực gia đình tuân thủ pháp luật và đảm bảo an toàn trong quá trình tố cáo?', 'a': '### Hỗ trợ nạn nhân bạo lực gia đình tuân thủ pháp luật\n\nĐể hỗ trợ các nạn nhân bạo lực gia đình trong việc tuân thủ pháp luật và đảm bảo an toàn khi tố cáo, cần thực hiện một số biện pháp nhằm tạo ra môi trường an toàn và tin cậy cho họ. Theo quy định tại Luật Phòng, chống bạo lực gia đình, các cơ sở hỗ trợ nạn nhân có trách nhiệm cung cấp dịch vụ tư vấn pháp luật và tâm lý, cũng như hỗ trợ các nhu cầu thiết yếu để giúp nạn nhân dễ dàng tiếp cận với các quyền lợi hợp pháp của mình. \n\n#### Tư vấn và Hỗ trợ Pháp lý\nCác cơ quan và tổ chức chuyên trách cần tổ chức các chương trình tư vấn pháp luật miễn phí cho nạn nhân bạo lực gia đình. Điều này không chỉ bao gồm việc cung cấp thông tin về quyền lợi của họ mà còn hướng dẫn họ cách thức thực hiện các bước tố cáo đúng đắn. Nhân viên tư vấn pháp lý cũng nên đảm bảo giữ bí mật thông tin cá nhân của nạn nhân, tạ

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 3 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 98%|█████████▊| 123/125 [26:09<00:24, 12.26s/it]

{'q': 'Những nguyên tắc pháp lý nào cần được chú trọng khi tư vấn phân chia tài sản trong quá trình ly hôn?', 'a': 'Khi tư vấn về phân chia tài sản trong quá trình ly hôn, có một số nguyên tắc pháp lý quan trọng mà các chuyên gia pháp lý cần chú trọng. Những nguyên tắc này không chỉ giúp đảm bảo quyền lợi của cả hai bên mà còn tạo nên một quá trình phân chia tài sản công bằng và hợp lệ.\n\n### 1. Phân loại tài sản\nTheo luật pháp Việt Nam, tài sản trong hôn nhân được phân thành hai loại chính: tài sản chung và tài sản riêng. Tài sản chung là tài sản được hình thành và tích lũy trong thời gian vợ chồng sống chung, trong khi tài sản riêng gồm các tài sản mà một bên có trước khi kết hôn hoặc nhận được qua thừa kế, quà tặng. Việc xác định rõ ràng loại tài sản nào thuộc về ai rất quan trọng trong quá trình phân chia.\n\n### 2. Nguyên tắc công bằng\nLuật Hôn nhân và Gia đình năm 2014 quy định rằng tài sản chung của vợ chồng sẽ được chia công bằng khi ly hôn. Điều này có nghĩa là tất cả tài s

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 7 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|██████████| 125/125 [26:20<00:00, 12.65s/it]

{'q': 'Luật quy định ra sao về việc bảo vệ quyền lợi của trẻ em trong các trường hợp ly hôn, và làm thế nào để tư vấn phụ huynh đảm bảo quyền này?', 'a': '### Quy định về bảo vệ quyền lợi của trẻ em trong trường hợp ly hôn\n\nTheo **Luật Hôn nhân và Gia đình Việt Nam năm 2014**, việc bảo vệ quyền lợi của trẻ em trong tình huống ly hôn được quy định cụ thể. Luật nhấn mạnh rằng quyền và lợi ích hợp pháp của trẻ em cần phải được đảm bảo và tôn trọng, đặc biệt là trong các vụ việc giải quyết hôn nhân. Trong trường hợp cha mẹ ly hôn, quyền lợi của trẻ em sẽ được quyết định dựa trên các yếu tố liên quan đến sự phát triển tốt nhất của trẻ.\n\nLuật quy định rằng cha mẹ có nghĩa vụ chăm sóc, nuôi dưỡng và giáo dục con cái, với quyền bày tỏ ý kiến của trẻ em có từ bảy tuổi trở lên được xem xét trong các quyết định liên quan đến việc nuôi dưỡng. Nếu cha mẹ không thể thỏa thuận được về việc ai sẽ là người trực tiếp nuôi dưỡng con sau khi ly hôn, Tòa án sẽ căn cứ vào quyền lợi của trẻ để quyết định

In [20]:
len(LIGHTRAG_NAIVE_QA)

125

In [21]:
with open("LIGHTRAG_NAIVE_QA.json", "w", encoding="utf-8") as json_file:
    json.dump(LIGHTRAG_NAIVE_QA, json_file, ensure_ascii=False, indent=4)

# Kiểm tra các câu hỏi thực tế

## Đề thi trắc nghiệm, đúng sai

In [24]:
len(test_multi_choice)

32

In [26]:
print(test_multi_choice[5]['q'])

Khi con mấy tuổi thì khi ly hôn phải xem xét vào nguyện vọng của con là sống cùng bố hay mẹ?
A. Từ đủ 7 tuổi.
C. Từ đủ 9 tuổi.
B. 7 tuổi.
D. 9 tuổi.


In [42]:
PROMPT['multi_choice'] = '''
Hãy chọn câu trả lời đúng, và giải thích vì sao chọn phương án đó, và các phương án còn lại là sai.
Nếu bạn không có thông tin gì thì trả về rỗng.

Đầu ra:
{
    "a" : "<đáp án đúng>", (A hoặc B hoặc C hoặc D hoặc ĐÚNG hoặc SAI, tùy vào dạng câu hỏi)
    "e" " "<giải thích>"
}

Đây là câu hỏi:

<<q>>
'''

In [33]:
res_test = rag.query(test, param=QueryParam(mode="hybrid"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:kw_prompt result:


{
  "high_level_keywords": ["Ly hôn", "Nguyện vọng của con", "Tuổi của con"],
  "low_level_keywords": ["7 tuổi", "9 tuổi", "Phương án A", "Phương án B", "Phương án C", "Phương án D"]
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 60 entites, 91 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Global query uses 12 entites, 7 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [32]:
print(res_test)

```json
{
    "a": "A. Từ đủ 7 tuổi.",
    "e": "Đáp án A là đúng vì theo quy định của pháp luật, khi con từ đủ 7 tuổi trở lên thì Tòa án sẽ phải xem xét nguyện vọng của con. Các phương án còn lại không đúng bởi: \n- Phương án B (7 tuổi) không chính xác vì không chỉ rõ 'từ đủ', tạo nên hiểu lầm rằng 7 tuổi là giới hạn mà không nhắc đến yêu cầu về độ tuổi cụ thể. \n- Phương án C (từ đủ 9 tuổi) và D (9 tuổi) là sai vì quy định không đặt mức tuổi cho nguyện vọng xem xét là 9 tuổi mà là 7 tuổi."
}
```


In [35]:
print(res_test)

```json
{
    "a": "A. Từ đủ 7 tuổi.",
    "e": "Theo quy định tại Điều 84 của Luật Hôn nhân và Gia đình 2014, việc thay đổi người trực tiếp nuôi con phải xem xét nguyện vọng của con từ đủ 7 tuổi trở lên. Do đó, phương án A là đúng. Các phương án còn lại không chính xác vì chúng hoặc cung cấp độ tuổi không đúng với quy định của pháp luật (B và C), hoặc không chính xác về mức độ là từ đủ 7 tuổi (D không cung cấp động từ 'đủ' chính xác trong nội dung chứa độ tuổi)."
}
```


In [45]:
MULTI_CHOICE_HYBRID = []

In [ ]:
num_correct_answer = 0

In [46]:
for q in test_multi_choice:
    print(q['q'])

    a = {}
    a['q'] = q['q']

    query = PROMPT['multi_choice'].replace("<<q>>", a['q'])

    #print(query)

    res = rag.query(query, param=QueryParam(mode="hybrid"))

    a['a'] = res

    print("AI:")
    print(a['a'])

    print("GoldAnswer:")

    a['sga'] = q['a'] 

    print(a['sga'])

    MULTI_CHOICE_HYBRID.append(a)
    

Pháp luật quy định Nam nữ được kết hôn khi đạt độ tuổi nào?
A. Nam từ đủ 19 tuổi, nữ từ đủ 17 tuổi.
B. Nam từ 20 tuổi, nữ từ 18 tuổi.
C. Nam từ đủ 20 tuổi, nữ từ đủ 18 tuổi.
D. Cả 3 phương án trên đều sai


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:kw_prompt result:


{
  "high_level_keywords": ["Pháp luật", "Kết hôn", "Độ tuổi"],
  "low_level_keywords": ["Nam", "Nữ", "19 tuổi", "17 tuổi", "20 tuổi", "18 tuổi", "Cả 3 phương án", "Sai"]
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 60 entites, 106 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Global query uses 12 entites, 7 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AI:
{
    "a": "C",
    "e": "Đáp án C là đúng vì Luật Hôn nhân và Gia đình 2014 quy định rõ ràng rằng độ tuổi kết hôn tối thiểu là nam từ đủ 20 tuổi và nữ từ đủ 18 tuổi. Phương án A sai vì độ tuổi cho nam và nữ chưa đúng theo quy định. Phương án B (dù gần đúng) vẫn sai ở việc thiếu từ 'đủ', không chính xác về cách diễn đạt của điều luật."
}
GoldAnswer:
C
Pháp luật cấm kết hôn trong những trường hợp nào sau đây:

A. Giữa người đang có vợ và đang có chồng.

B. Người mất năng lực hành vi dân sự.

C. Người có dòng máu về trực hệ; có họ trong phạm vi ba đời

D. Tất cả các trường hợp trên đều bị cấm.


...

AI:
{
    "a": "ĐÚNG",
    "e": "Theo quy định của Luật Hôn nhân và Gia đình, người đang chấp hành án phạt tù không được kết hôn vì họ đang ở trong tình trạng không có năng lực hành vi dân sự. Việc kết hôn yêu cầu các bên phải có đầy đủ năng lực hành vi, tức là họ phải có khả năng tự quyết định và thực hiện nghĩa vụ pháp lý của mình. Do đó, người đang thụ án có thể không đáp ứng được điều kiện này."
}
GoldAnswer:
ĐÚNG
Vợ chồng có quyền ly hôn theo yêu cầu của một bên.  ĐÚNG hay SAI?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:kw_prompt result:


{
  "high_level_keywords": ["Ly hôn", "Quyền yêu cầu", "Vợ chồng"],
  "low_level_keywords": ["ĐÚNG", "SAI"]
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 60 entites, 66 relations, 4 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Global query uses 12 entites, 7 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AI:
{
    "a": "ĐÚNG",
    "e": "Luật Hôn nhân và Gia đình 2014 quy định rằng vợ chồng có quyền ly hôn theo yêu cầu của một bên, do đó câu hỏi này là đúng. Theo đó, một trong hai bên có thể yêu cầu ly hôn mà không cần sự đồng ý của bên kia."
}
GoldAnswer:
ĐÚNG


In [47]:
with open("MULTI_CHOICE_HYBRID.json", "w", encoding="utf-8") as json_file:
    json.dump(MULTI_CHOICE_HYBRID, json_file, ensure_ascii=False, indent=4)

In [48]:
MULTI_CHOICE_NAIVE = []

In [49]:
for q in test_multi_choice:
    print(q['q'])

    a = {}
    a['q'] = q['q']

    query = PROMPT['multi_choice'].replace("<<q>>", a['q'])

    #print(query)

    res = rag.query(query, param=QueryParam(mode="naive"))

    a['a'] = res

    print("AI:")
    print(a['a'])

    print("GoldAnswer:")

    a['sga'] = q['a'] 

    print(a['sga'])

    MULTI_CHOICE_NAIVE.append(a)

Pháp luật quy định Nam nữ được kết hôn khi đạt độ tuổi nào?
A. Nam từ đủ 19 tuổi, nữ từ đủ 17 tuổi.
B. Nam từ 20 tuổi, nữ từ 18 tuổi.
C. Nam từ đủ 20 tuổi, nữ từ đủ 18 tuổi.
D. Cả 3 phương án trên đều sai


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 3 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AI:
{
    "a": "C",
    "e": "Pháp luật quy định độ tuổi tối thiểu để nam nữ kết hôn là nam từ đủ 20 tuổi trở lên và nữ từ đủ 18 tuổi trở lên, theo Điều 8 Luật Hôn nhân và gia đình năm 2014. Phương án A và B đều không đúng vì quy định tuổi kết hôn phải lớn hơn hoặc bằng 20 đối với nam và 18 đối với nữ."
}
GoldAnswer:
C
Pháp luật cấm kết hôn trong những trường hợp nào sau đây:

A. Giữa người đang có vợ và đang có chồng.

B. Người mất năng lực hành vi dân sự.

C. Người có dòng máu về trực hệ; có họ trong phạm vi ba đời

D. Tất cả các trường hợp trên đều bị cấm.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 3 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AI:
{
    "a": "D",
    "e": "Pháp luật cấm kết hôn trong tất cả các trường hợp A, B và C. Cụ thể, kết hôn giữa người đang có vợ hoặc chồng (A) là vi phạm quy định về chế độ hôn nhân một vợ một chồng; kết hôn với người mất năng lực hành vi dân sự (B) không hợp pháp do người đó không có khả năng đưa ra quyết định; và việc kết hôn giữa những người có dòng máu trực hệ hoặc có họ trong phạm vi ba đời (C) cũng bị cấm bởi luật hôn nhân. Do đó, lựa chọn D là đúng vì nó bao gồm tất cả các trường hợp cấm kết hôn."
}
GoldAnswer:
D
Việc kết hôn phải được đăng ký với cơ quan nào?

A. Thôn, bản, khối phố.

B. UBND cấp xã của 01 trong hai bên nam,nữ

C. UBND cấp huyện của 01 tronghai bên nam, nữ.

D. Nhà thờ.




...

AI:
{
    "a": "SAI",
    "e": "Nam nữ sống chung như vợ chồng mà không có đăng ký kết hôn thì pháp luật không công nhận họ là vợ chồng. Theo quy định tại điểm b, khoản 3 - Nghị quyết 35/2000/QH-10 của Quốc hội, trường hợp này chỉ được coi là chung sống mà không có giá trị pháp lý là quan hệ vợ chồng."
}
GoldAnswer:
ĐÚNG
Người đang chấp hành án phạt tù không được kết hôn.  ĐÚNG hay SAI?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 3 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AI:
{
    "a": "ĐÚNG",
    "e": "Theo quy định của pháp luật, người đang chấp hành án phạt tù sẽ bị hạn chế một số quyền, trong đó có quyền kết hôn. Điều này nhằm đảm bảo tính nghiêm ngặt của việc chấp hành án phạt và tạo điều kiện cho người phạm tội có cơ hội tự cải tạo bản thân trong thời gian thi hành án."
}
GoldAnswer:
ĐÚNG
Vợ chồng có quyền ly hôn theo yêu cầu của một bên.  ĐÚNG hay SAI?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 4 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AI:
{
    "a" : "ĐÚNG",
    "e" : "Theo quy định của pháp luật Việt Nam, một bên vợ hoặc chồng có quyền yêu cầu Tòa án giải quyết ly hôn mà không cần sự đồng ý của bên kia. Điều này được nêu trong Luật Hôn nhân và Gia đình, cho thấy rằng quyền yêu cầu ly hôn không phụ thuộc vào sự đồng thuận của cả hai bên."
}
GoldAnswer:
ĐÚNG


In [50]:
with open("MULTI_CHOICE_NAIVE.json", "w", encoding="utf-8") as json_file:
    json.dump(MULTI_CHOICE_NAIVE, json_file, ensure_ascii=False, indent=4)

## Các câu hỏi tự luận

In [66]:
len(test_tuluan)

28

In [79]:
TULUAN_HYBRID= []

In [80]:
TULUAN_HYBRID

[]

In [81]:
for q in test_tuluan:
    print(q['q'])

    a = {}
    a['q'] = q['q']

    query = q['q']

    hy = rag.query(query, param=QueryParam(mode="hybrid"))

    a['hybrid'] = hy

    print(hy)
    print()


    a['a'] = q['a']

    TULUAN_HYBRID.append(a)


Theo Luật hôn nhân và gia đình năm 2014, những nguyên tắc cơ bản của chế độ hôn nhân và gia đình ở nước ta được quy định như thế nào?
## Nguyên Tắc Cơ Bản của Chế Độ Hôn Nhân và Gia Đình theo Luật Hôn Nhân và Gia Đình Năm 2014

Luật Hôn nhân và Gia đình năm 2014, có hiệu lực từ ngày 01/01/2015, đã quy định một cách rõ ràng các nguyên tắc cơ bản trong chế độ hôn nhân và gia đình tại Việt Nam. Những nguyên tắc này không chỉ bảo đảm quyền và nghĩa vụ của các bên trong quan hệ hôn nhân mà còn thể hiện sự bình đẳng và công bằng giữa các thành viên trong gia đình. Dưới đây là một số nguyên tắc cơ bản được quy định trong bộ luật này.

### 1. Nguyên Tắc Một Vợ Một Chồng

Luật quy định rằng chế độ hôn nhân tại Việt Nam là hôn nhân một vợ một chồng. Điều này nhấn mạnh rằng mỗi người chỉ có thể kết hôn với một người khác tại cùng một thời điểm, nhằm bảo vệ quyền lợi hợp pháp và đạo đức trong quan hệ hôn nhân.

### 2. Nguyên Tắc Bình Đẳng Giữa Vợ và Chồng

Luật Hôn nhân và Gia đình năm 2014 quy đị

In [85]:
len(TULUAN_HYBRID)

28

In [82]:
with open("TULUAN_HYBRID.json", "w", encoding="utf-8") as json_file:
    json.dump(TULUAN_HYBRID, json_file, ensure_ascii=False, indent=4)

In [86]:
TULUAN_NAIVE= []

In [87]:
for q in test_tuluan:
    print(q['q'])

    a = {}
    a['q'] = q['q']

    query = q['q']

    hy = rag.query(query, param=QueryParam(mode="naive"))

    a['naive'] = hy

    print(hy)
    print()


    a['a'] = q['a']

    TULUAN_NAIVE.append(a)

Theo Luật hôn nhân và gia đình năm 2014, những nguyên tắc cơ bản của chế độ hôn nhân và gia đình ở nước ta được quy định như thế nào?
Theo Luật Hôn nhân và Gia đình năm 2014, các nguyên tắc cơ bản của chế độ hôn nhân và gia đình ở Việt Nam được quy định tại Điều 2. Những nguyên tắc này nhằm đảm bảo sự phát triển văn hóa và xã hội trong bối cảnh của đất nước, và cũng bảo vệ quyền lợi và nghĩa vụ của các thành viên trong gia đình. Dưới đây là các nguyên tắc cụ thể:

1. **Hôn nhân tự nguyện và bình đẳng**: Hôn nhân phải được xây dựng trên cơ sở tự nguyện của cả hai bên, không chỉ yêu cầu sự đồng ý mà còn phải đảm bảo sự bình đẳng giữa vợ và chồng. Nguyên tắc này nhấn mạnh rằng không có ai bị ép buộc kết hôn và cả hai vợ chồng đều có quyền và nghĩa vụ ngang nhau trong gia đình.

2. **Sự tôn trọng sự đa dạng**: Luật pháp bảo vệ hôn nhân giữa các công dân thuộc các dân tộc, tôn giáo khác nhau, cũng như giữa người theo tôn giáo và người không theo tôn giáo. Sự đa dạng trong nền văn hóa và tín

In [88]:
len(TULUAN_NAIVE)

28

In [89]:
with open("TULUAN_NAIVE.json", "w", encoding="utf-8") as json_file:
    json.dump(TULUAN_NAIVE, json_file, ensure_ascii=False, indent=4)

In [100]:
len(test_tuvan)

14

In [106]:
TUVAN_HYBRID = []

In [108]:
TUVAN_HYBRID

[]

In [109]:
for q in test_tuvan:
    print(q['q'])

    a = {}
    a['q'] = q['q']

    query = q['q']

    query += " . Hãy nêu rõ tên các điều luật được sử dụng để trả lời câu hỏi và vì sao dùng nó"


    hy = rag.query(query, param=QueryParam(mode="hybrid"))

    a['hybrid'] = hy

    print(hy)
    print()


    a['a'] = q['a']

    TUVAN_HYBRID.append(a)



Vợ chồng tôi lấy nhau đã nhiều năm nhưng vẫn chưa có con. Năm 2015, vợ chồng tôi quyết định nhận con nuôi và đã thực hiện theo đúng quy định về việc nhận con nuôi. Đến nay, tôi được biết vợ chồng tôi có thể nhờ mang thai hộ. Vậy xin hỏi, vợ chồng tôi đã nhận con nuôi thì có nhờ người khác mang thai hộ được không?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:kw_prompt result:


{
  "high_level_keywords": ["Nhận con nuôi", "Mang thai hộ", "Quy định pháp luật"],
  "low_level_keywords": ["Năm 2015", "Vợ chồng", "Điều luật", "Thủ tục nhận con nuôi", "Điều kiện mang thai hộ"]
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 60 entites, 93 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Global query uses 12 entites, 7 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### Vấn đề mang thai hộ sau khi nhận con nuôi

Việc mang thai hộ trong trường hợp vợ chồng đã nhận con nuôi là một vấn đề pháp lý phức tạp, liên quan đến các quy định trong Luật Hôn nhân và Gia Đình, cũng như Luật Nuôi con nuôi tại Việt Nam. Dưới đây là điểm trả lời cụ thể cho câu hỏi của bạn.

#### 1. Quy định về mang thai hộ 

Theo quy định tại **Luật Hôn Nhân và Gia Đình năm 2014** (số 52/2014/QH13), việc mang thai hộ được điều chỉnh tại Điều 95. Điều luật này quy định rằng mang thai hộ vì mục đích nhân đạo được áp dụng cho các cặp vợ chồng không thể sinh con tự nhiên. Như vậy, trong trường hợp của bạn, với Điều 95, vợ chồng bạn có thể thực hiện việc mang thai hộ nếu đáp ứng các yêu cầu cụ thể trong điều luật này.

#### 2. Quy định về nuôi con nuôi

Theo **Luật Nuôi con nuôi năm 2010**, việc nhận con nuôi không cản trở quyền lợi của các bậc phụ huynh trong việc có con hoặc tiến hành các biện pháp nhằm có con cái, bao gồm cả việc mang thai hộ. Điều 26 của Luật này quy định về quyền v

...

Đối mặt với tình huống pháp lý liên quan đến việc chồng bạn đã vay tiền bằng hình thức giả chữ ký của bạn, có thể áp dụng một số điều luật trong Luật Hôn nhân và Gia đình và Bộ luật Hình sự của Việt Nam. Dưới đây, tôi sẽ phân tích các khía cạnh liên quan đến vấn đề của bạn.

### 1. Quyền yêu cầu chuộc lại sổ đỏ

Theo **Điều 32 của Luật Hôn nhân và Gia đình năm 2014**, việc quản lý tài sản chung giữa vợ và chồng cần có sự đồng thuận của cả hai bên, đặc biệt là đối với các giao dịch quan trọng như bất động sản. Vì vậy, nếu chồng bạn đã vay tiền mà không có sự đồng ý của bạn, bạn có quyền yêu cầu ngân hàng trả lại sổ đỏ, dựa trên cơ sở rằng việc vay tiền này không có sự đồng thuận hợp pháp từ bạn.

### 2. Khả năng kiện chồng vì giả chữ ký

Theo **Điều 174, Bộ luật Hình sự 2015**, hành vi giả mạo chữ ký của một người khác là hành vi phạm tội có thể bị xử lý hình sự. Trong trường hợp của bạn, nếu bạn có thể chứng minh rằng chữ ký của bạn trên giấy vay tiền là giả mạo và không có sự đồng ý c

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:kw_prompt result:


{
  "high_level_keywords": ["Ly hôn", "Chăm sóc con", "Trách nhiệm pháp lý", "Cơ quan can thiệp"],
  "low_level_keywords": ["Vợ chồng", "Người con chung", "Mâu thuẫn", "Đưa con đi", "Sự đồng ý", "Điều luật"]
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 60 entites, 72 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Global query uses 12 entites, 7 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## Giải đáp về quyền nuôi con và trách nhiệm của các bên trong ly hôn

Trong trường hợp của bạn, các vấn đề liên quan đến việc nuôi dưỡng cũng như quyền và nghĩa vụ của cha mẹ đối với con cái sẽ được quy định theo Luật Hôn nhân và Gia đình 2014, cụ thể là các khoản liên quan đến quyền nuôi con và nghĩa vụ của vợ chồng trong quy trình ly hôn.

### 1. Quyền Nuôi Con

Theo **Điều 81** của Luật Hôn nhân và Gia đình 2014:

- Khi xảy ra ly hôn, vợ chồng có quyền thỏa thuận về người trực tiếp nuôi con, nếu không thỏa thuận được thì Tòa án sẽ quyết định trên cơ sở lợi ích của trẻ em.
- Trong mọi trường hợp, quyền lợi và sự phát triển của trẻ em phải được ưu tiên trong các quyết định của Tòa án.

### 2. Trách nhiệm của Vợ bạn

Nếu vợ bạn đưa con đi mà không có sự đồng ý của bạn, điều này có thể bị xem là vi phạm quyền nuôi dưỡng của bạn. Cụ thể:

- **Điều 84** quy định rằng cha mẹ có nghĩa vụ và quyền ruột thừa về việc nuôi dưỡng và chăm sóc con cái.
- Việc vợ bạn tự ý đưa con đi mà không có sự

In [110]:
len(TUVAN_HYBRID)

14

In [111]:
with open("TUVAN_HYBRID_ver2.json", "w", encoding="utf-8") as json_file:
    json.dump(TUVAN_HYBRID, json_file, ensure_ascii=False, indent=4)

In [112]:
TUVAN_NAIVE = []

In [113]:
TUVAN_NAIVE

[]

In [114]:
for q in test_tuvan:
    print(q['q'])

    a = {}
    a['q'] = q['q']

    query = q['q']

    query += " . Hãy nêu rõ tên các điều luật được sử dụng để trả lời câu hỏi và vì sao dùng nó"

    hy = rag.query(query, param=QueryParam(mode="naive"))

    a['hybrid'] = hy

    print(hy)
    print()


    a['a'] = q['a']

    TUVAN_NAIVE.append(a)

Vợ chồng tôi lấy nhau đã nhiều năm nhưng vẫn chưa có con. Năm 2015, vợ chồng tôi quyết định nhận con nuôi và đã thực hiện theo đúng quy định về việc nhận con nuôi. Đến nay, tôi được biết vợ chồng tôi có thể nhờ mang thai hộ. Vậy xin hỏi, vợ chồng tôi đã nhận con nuôi thì có nhờ người khác mang thai hộ được không?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 4 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Trong trường hợp của bạn, vợ chồng bạn có thể nhờ người khác mang thai hộ, ngay cả khi đã nhận con nuôi. Điều này được quy định rõ ràng trong Luật Hôn nhân và Gia đình năm 2014 (Luật số 52/2014/QH13).

**1. Căn cứ pháp lý:**
- **Điều 95** của Luật Hôn nhân và Gia đình 2014 quy định về điều kiện mang thai hộ vì mục đích nhân đạo. Theo điều này, việc mang thai hộ phải được thực hiện trên cơ sở tự nguyện của các bên và với các tiêu chí rõ ràng. Luật không yêu cầu các cặp vợ chồng đã nhận con nuôi không được nhờ người khác mang thai hộ.
- **Điều 94** cũng khẳng định rằng con sinh ra trong trường hợp mang thai hộ vì mục đích nhân đạo sẽ được xác định là con chung của vợ chồng nhờ mang thai hộ kể từ thời điểm con được sinh ra.

**2. Phân tích:**
Tuy đã nhận con nuôi, vợ chồng bạn vẫn có quyền nhờ người khác mang thai hộ mà không bị hạn chế theo quy định của pháp luật. Theo Điều 95, các bậc cha mẹ có quyền nhờ người mang thai hộ, với điều kiện xác nhận từ cơ quan y tế về việc vợ không thể man

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 3 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Theo quy định của pháp luật Việt Nam, việc kết hôn giữa hai cá nhân có thể được coi là hợp pháp chỉ khi hai bên đáp ứng đầy đủ các điều kiện theo quy định của Luật Hôn nhân và Gia đình năm 2014. Cụ thể, Điều 8 của luật này nêu rõ các điều kiện cần thiết cho việc kết hôn, trong đó có việc cấm kết hôn khi mà một trong hai bên đang có vợ hoặc chồng khác. 

1. **Điều 8 - Điều kiện kết hôn**: Theo Điều này, để được kết hôn hợp pháp, nam phải từ đủ 20 tuổi trở lên, nữ từ đủ 18 tuổi và việc kết hôn phải dựa trên quyết định tự nguyện của cả hai bên, không bị mất năng lực hành vi dân sự, và không thuộc một trong các trường hợp cấm kết hôn. Một trong những trường hợp cấm kết hôn được liệt kê là việc **kết hôn với người đã có vợ hoặc chồng**, điều mà bạn đang thực hiện khi bạn đã có hai đám cưới mà chưa đăng ký kết hôn với ai.

2. **Điều 5 - Các trường hợp cấm kết hôn**: Điều này quy định rõ ràng về các trường hợp bị cấm trong kết hôn, bao gồm việc lợi dụng quyền về hôn nhân để có những mục đích 

...

Chào bạn! Trong trường hợp của bạn, có thể áp dụng các quy định trong Bộ luật Hình sự 2015 sửa đổi bổ sung năm 2017 liên quan đến tội vi phạm chế độ một vợ một chồng.

### Điều luật liên quan

1. **Điều 182 - Tội vi phạm chế độ một vợ, một chồng**:
   Theo khoản 1, người nào đang có vợ hoặc chồng mà kết hôn hoặc chung sống như vợ chồng với người khác, hoặc người chưa có vợ hoặc chồng mà chung sống như vợ chồng với người mà mình biết rõ là đang có vợ, có chồng, sẽ bị xử phạt tùy thuộc vào mức độ vi phạm. 

2. **Điều 147 - Tội vi phạm chế độ một vợ một chồng**:
   Theo khoản 2, nếu hành vi vi phạm đó làm cho vợ, chồng hoặc con của một trong hai bên tự sát hoặc đã có quyết định của Tòa án hủy việc kết hôn mà vẫn duy trì quan hệ thì hình phạt có thể từ 6 tháng đến 3 năm tù.

### Giải thích quy định

Trong trường hợp của bạn, anh A đã có hành vi chung sống với chị B trong khoảng thời gian mà anh đã đăng ký kết hôn với bạn. Hành vi này vi phạm điều cấm về chế độ một vợ một chồng được quy địn

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 3 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Trường hợp của bạn liên quan đến nhiều vấn đề pháp lý khác nhau, bao gồm quyền sở hữu tài sản, hành vi lừa đảo và trách nhiệm hình sự. Dưới đây, tôi sẽ trả lời từng vấn đề một cách chi tiết:

### Quyền yêu cầu chuộc lại giấy chứng nhận quyền sử dụng đất

Theo quy định của Luật Đất đai năm 2013, nếu sổ đỏ (giấy chứng nhận quyền sử dụng đất) đang nằm trong tay ngân hàng và việc vay tiền không được sự đồng ý của bạn thì bạn hoàn toàn có quyền yêu cầu ngân hàng trả lại sổ đỏ cho bạn. Tuy nhiên, điều này sẽ phụ thuộc vào các điều khoản của hợp đồng tín dụng mà chồng bạn đã ký với ngân hàng.

### Hành vi giả mạo chữ ký

1. **Về việc kiện chồng**: Bạn có quyền khởi kiện chồng mình vì hành vi giả mạo chữ ký của bạn. Theo Điều 141 Bộ luật Hình sự 2015, hành vi này có thể được xem là tội "Lừa đảo chiếm đoạt tài sản". Chồng bạn sẽ bị truy cứu trách nhiệm hình sự nếu có đủ yếu tố cấu thành tội phạm.

2. **Người giả mạo chữ ký**: Nếu người đó đã hợp tác cùng chồng bạn để thực hiện việc giả mạo chữ 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Truncate 60 to 3 chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Trong tình huống của bạn, việc vợ bạn tự ý dẫn con đi mà không có sự đồng ý của bạn là một vấn đề pháp lý nghiêm trọng, và bạn có quyền yêu cầu sự can thiệp từ các cơ quan chức năng. Dưới đây là một số điều luật và giải pháp mà bạn có thể tham khảo.

### Quyền và Nghĩa vụ của Cha Mẹ

1. **Điều 81 Luật Hôn nhân và Gia đình 2014**: Điều này xác định rằng cha mẹ có trách nhiệm nuôi dưỡng và chăm sóc con cái. Trong trường hợp có tranh chấp về quyền nuôi con, phải xem xét quyền lợi của trẻ em trước hết và ưu tiên quy định bé sống cùng với cha hoặc mẹ. Việc dẫn trẻ đi mà không có sự đồng ý từ cha có thể được coi là vi phạm quyền nuôi dưỡng và chăm sóc của phụ huynh.

2. **Điều 84 Luật Hôn nhân và Gia đình 2014**: Điều này quy định những vấn đề liên quan đến nơi ở của trẻ em sau ly hôn. Trong thời gian chưa ly hôn, những quyết định liên quan đến việc ở của trẻ đều cần có sự đồng thuận của cả hai bên.

### Can Thiệp và Giải Pháp

Bạn có thể tìm sự trợ giúp từ cơ quan công an hoặc UBND xã/phườn

In [115]:
len(TUVAN_NAIVE)

14

In [116]:
with open("TUVAN_NAIVE_ver2.json", "w", encoding="utf-8") as json_file:
    json.dump(TUVAN_NAIVE, json_file, ensure_ascii=False, indent=4)